In [21]:
!pip install -q kaggle

In [29]:
!mkdir ~/.kaggle # creating a kaggle directory

The syntax of the command is incorrect.


In [23]:
pwd

'D:\\Projects\\Fundus Image analysis for early detection of diabetic retinopathy\\Fundus Image analysis for early detection of diabetic retinopathy\\Kaggle'

In [24]:
import os
filenames=os.listdir('D:\\Projects\\Fundus Image analysis for early detection of diabetic retinopathy\\Fundus Image analysis for early detection of diabetic retinopathy\\Kaggle')

In [16]:
cd D:\Projects\Fundus Image analysis for early detection of diabetic retinopathy\Fundus Image analysis for early detection of diabetic retinopathy\Kaggle

D:\Projects\Fundus Image analysis for early detection of diabetic retinopathy\Fundus Image analysis for early detection of diabetic retinopathy\Kaggle


In [25]:
ls

 Volume in drive D is DATA
 Volume Serial Number is C237-28C8

 Directory of D:\Projects\Fundus Image analysis for early detection of diabetic retinopathy\Fundus Image analysis for early detection of diabetic retinopathy\Kaggle

28-02-2023  14:34    <DIR>          #
28-02-2023  14:34    <DIR>          .
27-01-2023  17:35    <DIR>          ..
28-02-2023  14:34    <DIR>          ~
28-02-2023  14:34    <DIR>          a
28-02-2023  14:34    <DIR>          creating
28-02-2023  14:34    <DIR>          directory
16-11-2022  10:49    <DIR>          Kaggle
09-08-2022  18:23                72 kaggle.json
               1 File(s)             72 bytes
               8 Dir(s)  265,853,493,248 bytes free


In [26]:
!cp kaggle.json ~\.Kaggle\  # copying json file to folder

'cp' is not recognized as an internal or external command,
operable program or batch file.


In [27]:
!chmod 600 ~\.Kaggle\kaggle.json  # changing the permissions to json

'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [28]:
!kaggle datasets download -d arbethi/diabetic-retinopathy-level-detection

^C


In [6]:
!unzip diabetic-retinopathy-level-detection.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.xception import Xception, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [8]:
imageSize = [299, 299]

trainPath = r"/content/preprocessed dataset/preprocessed dataset/training"

testPath = r"/content/preprocessed dataset/preprocessed dataset/testing"

In [9]:
xception = Xception(input_shape=imageSize + [3], weights='imagenet',include_top=False)

In [10]:
# don't train existing weights
for layer in xception.layers:
  layer.trainable = False

In [11]:
# our layers - you can add more if you want
x = Flatten()(xception.output)

In [12]:
prediction = Dense(5, activation='softmax')(x)

In [13]:
# create a model object
model = Model(inputs=xception.input, outputs=prediction)

In [14]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [15]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [17]:
training_set = train_datagen.flow_from_directory('/home/wsuser/work/kaggle/preprocessed dataset/preprocessed dataset/training',
                                                 target_size = (299, 299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/testing',
                                            target_size = (299, 299),
                                            batch_size = 32,
                                            class_mode = 'categorical')

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/preprocessed dataset/preprocessed dataset/training'

In [ ]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=30,
  steps_per_epoch=len(training_set)//32,
  validation_steps=len(test_set)//32
)

In [ ]:
model.save('Updated-Xception-diabetic-retinopathy.h5')